## Tassi di Mortalità

In [1]:
# Import librerie per analisi dati (Pandas) e dati Istat
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import istat
import jsonstat

# cache dir per velocizzare analisi in locale
cache_dir = os.path.abspath(os.path.join("..", "tmp/od_la_grande_fuga", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  # lingua italiano
# print("cache_dir is '{}'".format(istat.cache_dir()))

0

In [2]:
dir_df = os.path.join(os.path.abspath(''),'stg')

In [3]:
istat_area_pop = istat.area('POP')

In [4]:
#istat_area_pop.datasets()

In [5]:
istat_dataset_popolazione = istat_area_pop.dataset('DCIS_DECESSI')
istat_dataset_popolazione

nr,name,nr. values,values (first 3 values)
0,Territorio,140,"1:'Italia', 3:'Nord', 4:'Nord-ovest' ..."
1,Tipo dato,9,"4:'morti', 65:'tasso di mortalità (per mille abitanti)', 264:'tasso specifico di mortalità per età (per mille abitanti)' ..."
2,Sesso,3,"1:'maschi', 2:'femmine', 3:'totale' ..."
3,Età,21,"6:'fino a 4 anni', 9:'5-9 anni', 20:'10-14 anni' ..."
4,Stato civile,7,"1:'nubile/celibe', 2:'coniugata/o', 3:'divorziata/o' ..."
5,Paese di cittadinanza,132,"83:'Angola', 101:'Burkina Faso (ex Alto Volta)', 106:'Benin (ex Dahomey)' ..."
6,Paese di nascita,157,"1:'Afghanistan', 3:'Albania', 4:'Algeria' ..."
7,Titolo di studio,7,"3:'licenza di scuola elementare, nessun titolo di studio', 4:'licenza di scuola media', 7:'diploma' ..."
8,Tempo,5,"2110:'2011', 2129:'2012', 2148:'2013' ..."


In [6]:
# istat_dataset_popolazione.dimensions()

In [7]:
# Numero di Morti per Classi di Età nel 2015 Nord
spec_n = {
    "Territorio":3,                 
    "Tipo dato":4,            
    "Sesso":3, 
    "Paese di nascita":118,
    "Titolo di studio":12,
    "Stato civile":8,
    "Tempo": 2186
}

In [8]:
# Numero di Morti per Classi di Età nel 2015 Sud
spec_s = {
    "Territorio":89,                 
    "Tipo dato":4,            
    "Sesso":3, 
    "Paese di nascita":118,
    "Titolo di studio":12,
    "Stato civile":8,
    "Tempo": 2186
}

In [9]:
def get_df(spec, value):
    c = istat_dataset_popolazione.getvalues(spec)
    ds = c.dataset(0)
    df = ds.to_data_frame('Paese di cittadinanza')
    df.reset_index(level=0, inplace=True)
    df = df[df['Paese di cittadinanza']=='Italia']
    df.columns = ['Paese di cittadinanza','Eta', value]
    df=df[(df['Eta']=='fino a 4 anni') |
       (df['Eta']=='5-9 anni') |
       (df['Eta']=='10-14 anni') |
       (df['Eta']=='15-19 anni') |
       (df['Eta']=='20-24 anni') |
       (df['Eta']=='25-29 anni') |
       (df['Eta']=='30-34 anni') |
       (df['Eta']=='35-39 anni') |
       (df['Eta']=='40-44 anni') |
       (df['Eta']=='45-49 anni') |
       (df['Eta']=='50-54 anni') |
       (df['Eta']=='55-59 anni') |
       (df['Eta']=='60-64 anni') |
       (df['Eta']=='65-69 anni') |
       (df['Eta']=='70-74 anni') |
       (df['Eta']=='75-79 anni') |
       (df['Eta']=='80-84 anni') |
       (df['Eta']=='85-89 anni') 
      ]
    df = df.groupby('Eta',as_index=False)
    df = df.agg({value : np.sum})
    return df

In [10]:
df_n = get_df(spec_n, 'Nord')
df_s = get_df(spec_s, 'Sud')

In [11]:
df = pd.merge(df_s,df_n, how='inner', on=['Eta', 'Eta'])

In [12]:
df.Eta.replace(['fino a 4 anni','5-9 anni'], ['00-04 anni','05-09 anni'], inplace=True)

In [13]:
df = df.sort_values(by=['Eta'], ascending=[1])

In [14]:
df_filename = r'df_morte.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df.to_pickle(df_fullpath)

In [15]:
# writer = pd.ExcelWriter('Eta.xlsx')
# df.to_excel(writer,'eta')
# writer.save()